In [1]:
# -*- coding: utf-8 -*-

"""
@author:SEN1211 Group 10 

Python implementation of 
This code contains a Government class claculate societal risks and make adaptations
The CollectiveAdaptationModel model contains will initialize the model and advance, in time step 5, a flood will happen.
In addition, it includes code for a Model class required
to implement government's collective decision-making in an agent-based model.

Notice: Since this is an RBB, there are no networks or actual data contains. Meaning everything here are based on assumptions.
Due to the limitation of web coding, This RBB is also limited.
This RBB model works fine with data. An example of using real data, containing networks can be find in the following link:

https://github.com/Codekiller258/SEN1211-GROUP-10-RBB.git

"""



# Importing necessary libraries
import networkx as nx
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import geopandas as gpd
import rasterio as rs
import matplotlib.pyplot as plt
import random
import math
from shapely.geometry import Point
from shapely import contains_xy
import numpy as np
from shapely import prepare




class CollectiveAdaptationModel(Model):
    """
    The main model running the simulation. It should be able to sets up the network of government agents,
    simulates their behavior, and collects data. The network type can be adjusted based on study requirements.
    
    However, this is simplified.
    Becahse it can not be done in the web based coding, 
    as we cannot input data to see the comprehensive version checek the link above.
    """

    def __init__(self, 
                 seed = None,
                 number_of_government = 0, # number of government agents
                 #local APE
                 APE=random.uniform(0.0005, 0.001),
                 #local Affected population
                 affected_population= 94623,
                 #local Affected population
                 exposure_rate=0.1,
                 #FN_standard= C/(death_population^alpha) we make assupmtions on C and alpha
                 c=0.031383,
                 a=1.5,
                 ):
        
        super().__init__(seed = seed)
        
        # defining the variables and setting the values
        self.number_of_government = number_of_government  # Total number of household agents
        self.seed = seed
        self.APE=APE
        self.affected_population= affected_population
        #local Affected population
        self.exposure_rate=exposure_rate
        self.c=c
        self.a=a

        # set schedule for agents
        self.schedule = RandomActivation(self)  # Schedule for activating agents

        # create households and add to the scheduler
        for i in range(number_of_government):
            government = Government(unique_id=i, model=self)
            self.schedule.add(government)
            #self.grid.place_agent(agent=government, node_id=node)


            
       # Data collection setup to collect data, The RBB will check how many household agents are doing the related adaptations
        model_metrics = {
                        "total_collective_adaptations": self.total_adapted_governments,
                        "weak_collective_adaptations": self.total_weak_adapted_governments,
                        "strong_collective_adaptations":self.total_strong_adapted_governments
                        }
        
        agent_metrics = {
                        "Weakadaptation": "weak_collective_adaption",
                        "Strongadaptation": "strong_collective_adaption",
                        "location":"location",
                        }
        #set up the data collector 
        self.datacollector = DataCollector(model_reporters=model_metrics, agent_reporters=agent_metrics)
            
    
    
    def total_weak_adapted_governments(self):
        """Return the total number of government agent that have decided on the weak collective adaptions."""
        weak_adapted_countg = sum([1 for agent in self.schedule.agents if isinstance(agent, Government) and agent.weak_collective_adaption])
        return weak_adapted_countg
    
    def total_strong_adapted_governments(self):
        """Return the total number of government agent that have decided on the strong collective adaptions."""
        strong_adapted_countg = sum([1 for agent in self.schedule.agents if isinstance(agent, Government) and agent.strong_collective_adaption])
        return strong_adapted_countg
    
    def total_adapted_governments(self):
        """Return the total number of government agent that have decided on the collective adaptions."""
        adapted_countg = sum([1 for agent in self.schedule.agents if isinstance(agent, Government) and agent.strong_collective_adaption or agent.weak_collective_adaption])
        return adapted_countg
    
  

    def step(self):
        """
        introducing a shock: 
        at time step 5, there will be a global flooding.
        This will result in actual flood depth. Here, we assume it is a random number
        between 0.5 and 1.2 of the estimated flood depth. In your model, you can replace this
        with a more sound procedure (e.g., you can devide the floop map into zones and 
        assume local flooding instead of global flooding). The actual flood depth can be 
        estimated differently
        """
        
        #At the beginning, set the value to the parameters
        if self.schedule.steps == 0:
            for agent in self.schedule.agents:
                agent.APE_in_own_node= self.APE
                agent.affected_population_in_own_node= self.affected_population
                agent.exposure_rate_in_own_node=self.exposure_rate
                agent.c=self.c
                agent.a=self.a
        
        if self.schedule.steps == 5:
            for agent in self.schedule.agents:
                # Calculate the actual flood depth as a random number between 0.5 and 1.2 times the estimated flood depth
                agent.flood_depth_actual = random.uniform(0.5, 1.2) * agent.flood_depth_own
  
        # Collect data and advance the model by one step
        self.datacollector.collect(self)
        self.schedule.step()
        
class Government(Agent):
    """
    A government agent in this RBB model will in charge of doing weak and strong adaptions, the weak adaption is information management,
    and the strong adaption is providing subsidies for the householders. This is general, users can change whatever they want accoding to 
    their needs.
    """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        #For the RBB, the government have two attribute, provide information and doing subsidies, both initial state is false
        self.weak_collective_adaption = False 
        self.strong_collective_adaption = False

         # Each node have an attribute related to the inputs APE, affected_population, exposure_rate, death_rate
        #
       
        self.APE_in_own_node= random.uniform(0.0005, 0.001)
        self.affected_population_in_own_node= 94623
        self.exposure_rate_in_own_node=0.1
        self.c=0.031383
        self.a=1.5
        
        #self.flood_depth_own = get_flood_depth(corresponding_map=model.flood_map, location=self.location, band=model.band_flood_img)
        #The above one can set the data from map, we donot have now, so we will use a simple one as follow.
        
        self.flood_depth_own = random.uniform(0, 3.5)
        
        if self.flood_depth_own < 0:
            self.flood_depth_own = 0
        self.death_rate_in_own_node= 0.655 * 0.001 * math.exp(1.16 * self.flood_depth_own)

        
    def calculate_FNstandard(self):
     """
    This function is to calculate the FN curve.
    
    Parameters
    ----------
    APE: annual probability of exceedance(years), we don't have data, have to make assumptions. For 200 years, we assume it was 1/200, i.e. 0.5%
    affected_population: the number of population affected in the flooded area. This can be counted according to the network?
    exposure_rate: rate of affected_population who are exposured to the flood, we can make assumptions, 10% for instance.
    death_rate: rate of deaths who are exposured to the flood, we can make assumptions, 0.0325% for instance. This can be linked to damage factors.

    Returns
    -------
    FN_standard: return to the FN standard value with the corresponding death population
    """
     self.exposure_population = self.affected_population_in_own_node * self.exposure_rate_in_own_node
     self.death_population = self.death_rate_in_own_node * self.exposure_population
     #FN_standard= C/(death_population^alpha) we make assupmtions on C and alpha
     self.FN_standard_own= self.c/(self.death_population**self.a)
     return self.FN_standard_own
    
    def calculate_FNvalue(self):
     """
    This function is to calculate the FN value.
    Due to lot of uncertainties, we will make it as simple as possible
    Parameters
    ----------
    APE: annual probability of exceedance(years), we don't have data, have to make assumptions. For 200 years, we assume it was 1/200, i.e. 0.5%

    Returns
    -------
    FN_value: return to the FN value with the corresponding APE
    """
    # FN=1-Rn/(k+1) here Rn is the rank of the flood event, k is the event per year, we assume it is the APE
     self.FN_value_own =1-1/(self.APE_in_own_node+1) 
     return self.FN_value_own
    
    #Functions to determine what to use    
    def step(self):
        # Logic for adaptation based on the FN value and FN standard
        #The user can change the logic as they like to fit their own model
        # Standard should have two, to determine whether to use strong or weak adaptions
        self.calculate_FNstandard()
        self.calculate_FNvalue()
    
        if self.FN_value_own > 0.6*self.FN_standard_own and random.random() < 0.2:
            self.weak_collective_adaption = True  # government agent decide on collective adaption strategies
        if self.weak_collective_adaption == True and self.FN_value_own > self.FN_standard_own and random.random() < 0.4:
            self.strong_collective_adaption = True
            

            
#Notice, Notice, Notice! these code in this cell is already simplified, the implimentation of our RBB is more thorough.
#To get the more comprehensive version of RBB, which contains Visualization, Network, Real flood data. See:
# https://github.com/Codekiller258/SEN1211-GROUP-10-RBB.git

#To run the simple version of RBB, you can change some of the parameters in the model as you like. 
# For the parameter input of flood data, go the the Government class, check the parameter self.flood_depth_own. 
# For this simple model, run the following code for an example
# You can see the output with governments' decision making overtime and an overall decision-making data with the example parameter.
model = CollectiveAdaptationModel(number_of_government=50,APE=random.uniform(0.0005, 0.001),
                                  affected_population= 94623,exposure_rate=0.1,c=0.031383,a=1.5,)
for step in range(20):
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()
model_data = model.datacollector.get_model_vars_dataframe()
print(agent_data)
print(model_data)

              Weakadaptation  Strongadaptation location
Step AgentID                                           
0    0                 False             False     None
     1                 False             False     None
     2                 False             False     None
     3                 False             False     None
     4                 False             False     None
...                      ...               ...      ...
19   45                 True             False     None
     46                 True              True     None
     47                 True              True     None
     48                 True              True     None
     49                 True              True     None

[1000 rows x 3 columns]
    total_collective_adaptations  weak_collective_adaptations  \
0                              0                            0   
1                              8                            8   
2                             15                    